# Doesn't work yet!
## PM if I get lazy and dont implement it for too long!
## LSTM state encoder

P.S. This snippet uses the library version of the learning function, you can see the visualization in the tensorboard


**And aslo this lstm state encoder is experemental. Recnn is an RL library, you are expected to do dataloading by yourself**

### **Note on this tutorials:**
**They mostly contain low level implementations explaining what is going on inside the library.**

**Most of the stuff explained here is already available out of the box for your usage.**

In [2]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import torch_optimizer as optim

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline


# == recnn ==
import sys
sys.path.append("../../")
import recnn

cuda = torch.device('cuda')

# ---
frame_size = 10
batch_size = 25
n_epochs   = 100
plot_every = 30
step       = 0
# --- 

tqdm.pandas()

In [3]:
state_encoder = nn.LSTM(129, 256, batch_first=True).to(cuda)

# embeddgings: https://drive.google.com/open?id=1EQ_zXBR3DKpmJR3jBgLvt-xoOvArGMsL
dirs = recnn.data.env.DataPath(
    base="../../../data/",
    embeddings="embeddings/ml20_pca128.pkl",
    ratings="ml-20m/ratings.csv",
    cache="cache/frame_env.pkl", # cache will generate after you run
    use_cache=True
)
env = recnn.data.env.FrameEnv(dirs, frame_size, batch_size)

In [4]:
def run_tests():
    batch = next(env.test_batch())
    loss = ddpg_update(batch, params, nets, optimizer,
                       cuda, debug, writer, step=step, learn=False)
    return losses

In [5]:
# === ddpg settings ===

params = {
    'gamma'      : 0.99,
    'min_value'  : -10,
    'max_value'  : 10,
    'policy_step': 10,
    'soft_tau'   : 0.001,
    
    'policy_lr'  : 1e-5,
    'value_lr'   : 1e-5,
    'actor_weight_init': 54e-2,
    'critic_weight_init': 6e-1,
}

# === end ===

In [6]:

value_net  = recnn.nn.Critic(256, 128, 256, params['critic_weight_init']).to(cuda)
policy_net = recnn.nn.Actor(256, 128, 256, params['actor_weight_init']).to(cuda)

target_value_net = recnn.nn.Critic(256, 128, 256).to(cuda)
target_policy_net = recnn.nn.Actor(256, 128, 256).to(cuda)

target_policy_net.eval()
target_value_net.eval()


recnn.utils.soft_update(value_net, target_value_net, soft_tau=1.0)
recnn.utils.soft_update(policy_net, target_policy_net, soft_tau=1.0)

pm = list(policy_net.parameters()) + list(state_encoder.parameters())
value_optimizer = optim.RAdam(value_net.parameters(),
                              lr=params['value_lr'], weight_decay=1e-2)
policy_optimizer = optim.RAdam(pm, lr=params['policy_lr'] , weight_decay=1e-2)

nets = {
    'value_net': value_net,
    'target_value_net': target_value_net,
    'policy_net': policy_net,
    'target_policy_net': target_policy_net,
}

optimizer = {
    'policy_optimizer': policy_optimizer,
    'value_optimizer':  value_optimizer
}

debug = {}
writer = SummaryWriter(log_dir='../../runs')

In [7]:
step = 0
for batch in tqdm(env.train_batch()):
        loss = recnn.nn.update.ddpg_update(batch, params, nets, optimizer,
                       cuda, debug, writer, step=step)
        #if step % 10 == 0:
        #    run_tests()

KeyboardInterrupt: 